In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

Preprocessing

In [10]:
def training_processing(trainfile, num):
    # Read the training data from CSV file
    # File has no column names and values are separated by spaces
    df = pd.read_csv(trainfile, sep=" ", header=None)
    
    # Drop columns 26 and 27 from the dataframe
    # They don't contain relevant information and many values are NANs
    df = df.drop([26, 27], axis=1)
    
    # Rename the columns:
    # - first two columns are "ID" and "Cycle"
    # - next three columns represent operational settings: "OpSet1", "OpSet2", and "OpSet3"
    # - remaining columns (from 4 to 25) represent sensor measurements: "SensorMeasure1" to "SensorMeasure21"
    df.columns = ["ID", "Cycle"] + [f"OpSet{i}" for i in range(1, 4)] + [f"SensorMeasure{i}" for i in range(1, 22)]
    
    # Calculate end-of-life (EOL) cycle for each engine (each unique "ID")
    # The 'EOL' column contains the maximum cycle number for each engine
    df['EOL'] = df.groupby('ID')['Cycle'].transform('max')
    
    # Calculate Remaining Useful Life (RUL) for each cycle by subtracting current cycle num from the EOL
    # This gives the number of cycles remaining before the engine reaches the end of its life
    df['RUL'] = df['EOL'] - df['Cycle']
    
    # Add 1000 times the number of the current dataset to the IDs
    # This is to distinguish between engines from different datasets
    df['ID'] = df['ID'].apply(lambda x: int(num * 1000 + x))
    
    # Drop 'EOL' column since it's no longer needed and return processed dataframe
    return df.drop(columns=['EOL'])


In [11]:
def testing_processing(testfile, rulfile, num):
    # Read the testing data from CSV file
    # File has no column names and values are separated by spaces
    df = pd.read_csv(testfile, sep=" ", header=None)
    
    # Drop columns 26 and 27 from the dataframe
    # They don't contain relevant information and many values are NANs
    df = df.drop([26, 27], axis=1)
    
    # Rename the columns:
    # - The first two columns are "ID" and "Cycle"
    # - The next three columns represent operational settings: "OpSet1", "OpSet2", and "OpSet3"
    # - The remaining columns (from 4 to 25) represent sensor measurements: "SensorMeasure1" to "SensorMeasure21"
    df.columns = ["ID", "Cycle"] + [f"OpSet{i}" for i in range(1, 4)] + [f"SensorMeasure{i}" for i in range(1, 22)]
    
    # Read the Remaining Useful Life (RUL) values from the parameter RUL file
    rul_list = pd.read_csv(rulfile, header=None)[0].values
    
    # Calculate the total number of cycles for each engine in the test data
    df['NumCycles'] = df.groupby('ID')['Cycle'].transform('count')
    
    # Calculate the RUL for each engine in the test data:
    # - The RUL for each cycle is the sum of the RUL from the RUL file and the difference between the total cycle count and the current cycle number
    df['RUL'] = rul_list[df['ID'].astype(int) - 1] + df['NumCycles'] - df['Cycle']
    
    # Adjust the 'ID' values by adding 1000 times the dataset number
    # This is to ensure unique IDs across different datasets
    df['ID'] = df['ID'].apply(lambda x: int(num * 1000 + x))
    
    # Drop the 'NumCycles' column since it's no longer needed and return processed dataframe
    return df.drop(columns=['NumCycles'])

In [12]:
# Initialize empty DataFrames for training and testing data
training_set, testing_set = pd.DataFrame(), pd.DataFrame()

# Loop through each of the four datasets (FD001 to FD004)
for num in range(1, 5):
    # Process the training/testing data from the current dataset and concatenate it to the overall training/testing set
    training_set = pd.concat([training_set, training_processing(f"cmapss\\train_FD00{num}.txt", num)], ignore_index=True)
    testing_set = pd.concat([testing_set, testing_processing(f"cmapss\\test_FD00{num}.txt", f"cmapss\\RUL_FD00{num}.txt", num)], ignore_index=True)

# Separate the 'ID' and 'RUL' columns from the rest of the training and testing sets
# 'ID' and 'RUL' are not needed for feature scaling and will be added back later
trainingIDs, testingIDs = training_set.pop('ID'), testing_set.pop('ID')
trainingRULs, testingRULs = training_set.pop('RUL'), testing_set.pop('RUL')

# Store the original column names for later use after scaling
original_columns = training_set.columns

# Initialize a MinMaxScaler to scale the data (sensor measurements and operational settings)
data_scaler = MinMaxScaler()

# Scale the training data and convert it back to a DataFrame with the original column names
training_set = pd.DataFrame(data_scaler.fit_transform(training_set), columns=original_columns)

# Scale the testing data using the same scaler (fit on training data) and convert it back to a DataFrame
testing_set = pd.DataFrame(data_scaler.transform(testing_set), columns=original_columns)

# Initialize a MinMaxScaler to scale the RUL values separately
rul_scaler = MinMaxScaler()

# Scale the training RULs and reshape them to a 2D array required by the scaler
training_rul_scaled = pd.DataFrame(rul_scaler.fit_transform(trainingRULs.values.reshape(-1, 1)))

# Scale the testing RULs using the same scaler (fit on training RULs) and reshape to a DataFrame
testing_rul_scaled = pd.DataFrame(rul_scaler.transform(testingRULs.values.reshape(-1, 1)))

# Concatenate the 'ID' column, scaled feature data, and scaled RULs back together for the training/testing sets
training_set = pd.concat([trainingIDs, training_set, training_rul_scaled], axis=1)
testing_set = pd.concat([testingIDs, testing_set, testing_rul_scaled], axis=1)

# Save the processed training/testing set to CSV files in the "processed" directory
training_set.to_csv(f"processed\\training_set", index=False)
testing_set.to_csv(f"processed\\testing_set", index=False)

In [13]:
def create_padded_sequences(data_set):
    # Group the dataset by 'ID', keeping only the feature columns (excluding 'ID')
    grouped = data_set.groupby('ID')[list(data_set.columns)[1:]]
    
    # Initialize an empty list to store sequences for each engine (each unique 'ID')
    sequences = []
    
    # Iterate over each group (each engine's data)
    for _, group in grouped:
        # Convert the group's data to a list of lists (each sublist represents a row)
        sequence = group.values.tolist()
        
        # Append the sequence (all cycles for one engine) to the sequences list
        sequences.append(sequence)

    # Determine the maximum length of the sequences (i.e., the engine with the most cycles)
    # This length will be used to pad (add 0s) all sequences to the same length
    #! maxlen for training_set is 543 and maxlen for testing_set is 486.
    #! Because the model is not capable of handling a variable maxlen, a constant number is necessary.
    #! 545 was chosen because it's greater than both maxlens and for simplicity.
    maxlen = 545
    
    # Pad all sequences to the maximum length (maxlen)
    # Sequences shorter than maxlen will be padded with zeros at the end ('post' padding)
    # Sequences longer than maxlen will be truncated to maxlen ('post' truncating)
    # Truncating will not happen since every sequence is shorter than chosen maxlen
    # All elements are cast to 'float32' type
    padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post', dtype='float32')
    
    # Return the padded sequences as a NumPy array
    return padded_sequences

# Create padded sequences for the training/testing set
train_sequences = create_padded_sequences(training_set)
test_sequences = create_padded_sequences(testing_set)


# Save the training/testing padded_sequences to CSV files in the "processed" directory
with open(f"processed\\train_padded_sequences.txt", 'w') as f:
    for sequence in train_sequences:
        for row in sequence:
            f.write(' '.join(map(str, row)) + '\n')
        f.write('\n')
with open(f"processed\\test_padded_sequences.txt", 'w') as f:
    for sequence in test_sequences:
        for row in sequence:
            f.write(' '.join(map(str, row)) + '\n')
        f.write('\n')

In [14]:
# Extract features (X) and labels (y) from the training sequences
# The features (X) include all columns except the last one
# The last column is the target variable (RUL), which becomes the label (y)
x_train = train_sequences[:, :, :-1]
y_train = train_sequences[:, :, -1]

# Extract features (X) and labels (y) from the testing sequences
# The features (X) include all columns except the last one
# The last column is the target variable (RUL), which becomes the label (y)
x_test = test_sequences[:, :, :-1]
y_test = test_sequences[:, :, -1]

In [15]:
from tensorflow.keras.layers import Masking, LSTM, Bidirectional, Dense, Dropout, BatchNormalization

class LSTMModel(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.3, mask_value=0.0):
        super(LSTMModel, self).__init__()
        
        # Masking layer to ignore padded values (zeros) in the input sequences
        # `mask_value` specifies the value that should be ignored during training
        self.masking = Masking(mask_value=mask_value)

        # First LSTM layer: processes the input sequences and returns the full sequence
        # `hidden_dim` specifies the number of units in the LSTM
        # `return_sequences=True` ensures that the LSTM returns the entire sequence of outputs
        self.lstm1 = Bidirectional(LSTM(hidden_dim, return_sequences=True))

        # Second LSTM layer: processes the output of the first LSTM and returns the final output only
        # `return_sequences=False` ensures that only the last output of the sequence is returned
        self.lstm2 = Bidirectional(LSTM(hidden_dim, return_sequences=False))

        # Dropout layers: used to prevent overfitting by randomly setting a fraction of input units to 0 during training
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

        # Batch normalization layers: normalize the output of the LSTM layers to improve training stability
        # `training=training` ensures that batch normalization behaves differently during training and inference
        self.batch_norm1 = BatchNormalization()
        self.batch_norm2 = BatchNormalization()

        # Dense layer 1: fully connected layer with ReLU activation
        # `hidden_dim` is the number of output units
        # `kernel_regularizer` adds L2 regularization to prevent overfitting
        # L2 regularization helps prevent overfitting by adding a penalty proportional to the square of the weights' magnitude.
        self.dense1 = Dense(hidden_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))

        # Dense layer 2: another fully connected layer with ReLU activation
        # This layer reduces the dimensionality by half
        self.dense2 = Dense(hidden_dim // 2, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))

        # Output layer: final dense layer that outputs the prediction
        # `output_dim` specifies the number of output units
        # `activation='linear'` which is used for regression tasks, produces continuous values
        self.output_layer = Dense(output_dim, activation='linear')

    def call(self, inputs, training=False):
        # Apply masking to ignore padded values
        x = self.masking(inputs)

        # Pass the masked input through the first LSTM layer
        x = self.lstm1(x)

        # Apply dropout and batch normalization to the output of the first LSTM layer
        x = self.dropout1(x, training=training)
        x = self.batch_norm1(x, training=training)

        # Pass the normalized output through the second LSTM layer
        x = self.lstm2(x)

        # Apply dropout and batch normalization to the output of the second LSTM layer
        x = self.dropout2(x, training=training)
        x = self.batch_norm2(x, training=training)

        # Pass the normalized output through the first dense layer
        x = self.dense1(x)

        # Pass the output through the second dense layer
        x = self.dense2(x)

        # Return the final output from the output layer
        return self.output_layer(x)

In [16]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Hyperparameters
input_dim = x_train.shape[2]  # The number of features in the input data
hidden_dim = 128  # Number of units in the LSTM layers (hidden state dimensionality)
output_dim = 545  # Output dimensionality (RUL prediction for each cycle)
learning_rate = 0.001  # Initial learning rate for the optimizer
batch_size = 64  # Number of samples per gradient update
epochs = 50  # Number of epochs to train the model

# Model instantiation
# Initialize the LSTM model with the specified input, hidden, and output dimensions
model = LSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

# Compile the model
# Adam optimizer is used with the specified learning rate
# The loss function is mean absolute error (MAE), and we also track MAE and MSE as metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mae', metrics=['mae', 'mse'])

# Callbacks
# EarlyStopping: Stop training if the validation loss doesn't improve for 10 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# ReduceLROnPlateau: Reduce learning rate by a factor of 0.2 if validation loss doesn't improve for 5 epochs, with a minimum learning rate of 1e-6
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Save the model only if the validation loss improves; keeps best version of model, helps in selecting a model that generalizes well to unseen data
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

# Train the model
# Fit the model on the training data, using 20% of it for validation
# The training process will stop early if validation loss doesn't improve (early_stopping)
# Learning rate will be reduced if the model stops improving (reduce_lr)
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,  # Number of samples processed before the model's internal parameters are updated
    epochs=epochs,  # Number of complete passes through the training data
    validation_split=0.2,  # Fraction of training data used for validation
    callbacks=[early_stopping, reduce_lr, model_checkpoint],  # List of callbacks to apply during training
    verbose=1  # Verbosity mode (1 = progress bar)
)

# Evaluate the model
evaluation = model.evaluate(x_test, y_test, verbose=2)
print(f"Test Loss: {evaluation[0]:.4f}")  # Print the test loss (MAE)
print(f"Test MAE: {evaluation[1]:.4f}")   # Print the test MAE
print(f"Test MSE: {evaluation[2]:.4f}")   # Print the test MSE

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 96s 10s/step - loss: 2.6825 - mae: 0.2363 - mse: 0.0982 - val_loss: 2.2367 - val_mae: 0.1167 - val_mse: 0.0400 - learning_rate: 0.0010
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 98s 11s/step - loss: 2.1488 - mae: 0.1257 - mse: 0.0299 - val_loss: 1.8490 - val_mae: 0.1133 - val_mse: 0.0381 - learning_rate: 0.0010
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 108s 12s/step - loss: 1.7459 - mae: 0.0933 - mse: 0.0186 - val_loss: 1.5177 - val_mae: 0.1092 - val_mse: 0.0362 - learning_rate: 0.0010
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 109s 12s/step - loss: 1.4162 - mae: 0.0772 - mse: 0.0136 - val_loss: 1.2411 - val_mae: 0.1049 - val_mse: 0.0343 - learning_rate: 0.0010
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 106s 12s/step - loss: 1.1428 - mae: 0.0636 - mse: 0.0093 - val_loss: 1.0136 - val_mae: 0.1004 - val_mse: 0.0317 - learning_rate: 0.0010
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 109s 12s/step - loss: 0.9209 - mae: 0.0540 - mse: 0.0075 - val_loss: 0.8260 - val_mae: 0.0939 - val_mse: 0

In [17]:
mae_scaled = model.evaluate(x_test, y_test, verbose=0)[1]

# To convert MAE back to original scale:
mae_original = rul_scaler.inverse_transform(np.array([[mae_scaled]]))[0][0]

print(f"Original and Scaled MAE: {mae_original}, {mae_scaled}")

Original and Scaled MAE: 17.52151468396187, 0.03232751786708832
